<H1>FBN1: Marfan syndrome (Tiecke, 2001)</H1>
<p>Extract phenopackets from the clinical data in <a href="https://pubmed.ncbi.nlm.nih.gov/11175294/" target="__blank">Tiecke et al (2001)</a>.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
# last tested with pyphetools version 0.2.21

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199")
metadata.default_versions_with_hpo(version=hpo_version)

In [3]:
df = pd.read_table("input/tiecke_2001.tsv")

In [4]:
df

,Feature/finding,B7,B15,B22,B27,B35,B47,D1,D13,D27,D36,D54,D60
0,AgeLastExamination(years),13,16,32,35,10,3,50,28,35,11,33,8
1,Inheritance,fam.,spor.,fam.,NaN,fam.,fam.,fam.,fam.,fam.,fam.,spor.,spor.
2,Gender,f,f,m,f,f,f,m,f,m,f,m,f
3,HeightOver98P,+,–,+,–,+,+,+,–,+,+,+,+
4,PectusDeformity,car.,exc.,car.,asym.,car.,–,–,–,car.,–,exc.,–
5,WristAndThumbSign,+,+,+,+,+,±,–,NaN,NaN,+,+,+
6,Scoliosis,–,+,+,+,+,–,–,+,–,–,+,+
7,HyperextensibleJoints,–,–,–,–,+,+,+,NaN,NaN,+,–,+
8,Other,HP,HP;D,HP;D,DE,HP;D,HP,HP;D,–,HP;D,–,St,HP;D;En
9,EctopiaLentis,–,+,+,–,–,NaN,+,+,+,+,+,+


In [5]:
# convert to row-based format
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
#dft=dft.rename(columns = {'Feature/Finding':'Patient'})
#dft['patient_id'] = dft.index
dft['patient_id'] = dft.index
dft.head()

Feature/finding,AgeLastExamination(years),Inheritance,Gender,HeightOver98P,PectusDeformity,WristAndThumbSign,Scoliosis,HyperextensibleJoints,Other,EctopiaLentis,Myopia(diopters),DilationAscendingAorta(mm),AorticRootReplacement(age),MitralValve,OtherCV,UnusualFeatures,patient_id
B7,13,fam.,f,+,car.,+,–,–,HP,–,+(–2),+(38),–,MVP,–,–,B7
B15,16,spor.,f,–,exc.,+,+,–,HP;D,+,+,+(75),16,MR,AR,+,B15
B22,32,fam.,m,+,car.,+,+,–,HP;D,+,+(–10),+(50),32,MVP,–,–,B22
B27,35,NaN,f,–,asym.,+,+,–,DE,–,+(–2),+(40),34,MVP,AR,+,B27
B35,10,fam.,f,+,car.,+,+,+,HP;D,–,mild,+(28),28,MVP,–,–,B35


In [6]:
# Add in the "unusual features" not in the table from the text
dft.at['B15', 'UnusualFeatures'] = "Thin skin; Decreased body weight"
dft.at['B27', 'UnusualFeatures'] = "Ulnar deviation of finger; Narrow face; Dental malocclusion; Dural ectasia; Patent foramen ovale "


In [7]:
column_mapper_d = defaultdict(ColumnMapper)

In [8]:
items = {
      'HeightOver98P': ['Tall stature', 'HP:0000098'],
      'WristAndThumbSign': ['Arachnodactyly', 'HP:0001166'],
      'Scoliosis': ['Scoliosis', 'HP:0002650'],
      'HyperextensibleJoints': ['Joint hypermobility','HP:0001382'],
      'EctopiaLentis': ["Ectopia lentis", "HP:0001083"],  
}
item_column_mapper_d = hpo_cr.initialize_simple_column_maps(column_name_to_hpo_label_map=items, 
                                                            observed=['+','±'],
                                                            excluded='-')

# Transfer to column_mapper_d
for k, v in item_column_mapper_d.items():
    column_mapper_d[k] = v

In [9]:
option_d = {
    "car.": ["Pectus carinatum", "HP:0000768"],
    "exc.": ["Pectus excavatum", "HP:0000767"]
}
pectusMapper = OptionColumnMapper(concept_recognizer=hpo_cr ,option_d=option_d)
pectusMapper.preview_column(dft["PectusDeformity"])
column_mapper_d["PectusDeformity"] = pectusMapper

In [10]:
# Other
other_d = {
    "HP": ["High palate", "HP:0000218"],
    "D": ["Dolichocephaly", "HP:0000268"],
    "En": ["Deeply set eye", "HP:0000490"], # i.e., Enophthalmus
    "DE": ["Dural ectasia", "HP:0100775"],
    "St": ["Striae distensae", "HP:0001065"]
}
otherMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=other_d)
otherMapper.preview_column(dft["Other"])
column_mapper_d["Other"] = otherMapper

In [11]:
myopia_d ={
    "+(–2)" : "Mild myopia",
    "+": "Myopia",
    "+(–10)": "High myopia",
    "mild": "Mild myopia",
    "+(–13)": "High myopia",
    "+(–6)": "High myopia"
}
myopiaMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=myopia_d)
myopiaMapper.preview_column(dft["Myopia(diopters)"])
column_mapper_d["Myopia(diopters)"] = myopiaMapper

In [12]:
# DilationAscendingAorta(mm) -- all patients have this feature
aoDilMapper = ConstantColumnMapper(hpo_id="HP:0002616", hpo_label="Aortic root aneurysm")
aoDilMapper.preview_column(dft["DilationAscendingAorta(mm)"])
column_mapper_d["DilationAscendingAorta(mm)"] = aoDilMapper

In [13]:
mitral_d = {
    "MVP": "Mitral valve prolapse",
    "MR": "Mitral regurgitation",
    "MVrep": "Mitral regurgitation" # replacement because of severe MR 
}
mitralMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=mitral_d)
mitralMapper.preview_column(dft["MitralValve"])
column_mapper_d["MitralValve"] = mitralMapper

In [14]:
#OtherCV
cv_d = {
    "AR":"Aortic regurgitation",
    "Pneu": "Spontaneous pneumothorax"
}
othercvMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=cv_d)
othercvMapper.preview_column(dft["OtherCV"])
column_mapper_d["OtherCV"] = othercvMapper

In [15]:
# Variants -- modernized from the original publication to use HGVS
variant_d = {
    "B15":  "c.3037G>A",
    "B47": "c.3058A>G",
    "D54": "c.3082+1G>T", # corresponds to IVS24 +1 G to T, Skipping of exon 24 
    "D60":  "c.3037G>A",
    "B27": "c.3344A>G",
    "B35": "c.3713A>G",
    "D1":  "c.3898A>G",
    "B7": "c.3973G>T",
    "D36": "c.4217A>G",
    "D13": "c.4223G>T",
    "D27": "c.4615C>T",
    "B22": "c.4955G>A"
}

dft["Variants"] = variant_d
genome = 'hg38'
transcript='NM_000138.5' # FBN1
varMapper = VariantColumnMapper(assembly=genome,
                                column_name='Variants', 
                                transcript=transcript, 
                                default_genotype='heterozygous')

In [16]:
sexMapper = SexColumnMapper(male_symbol="m", female_symbol="f", column_name="Gender")
pmid = "PMID:11175294"
ageMapper = AgeColumnMapper.by_year("AgeLastExamination(years)")
encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="patient_id", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper,
                        metadata=metadata,
                        pmid=pmid)
omim_id = "OMIM:154700"
omim_label = "Marfan syndrome"
encoder.set_disease(disease_id=omim_id, label=omim_label)

In [17]:
individuals = encoder.get_individuals()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3973G>T/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3037G>A/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.4955G>A/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3344A>G/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3713A>G/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3058A>G/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3898A>G/NM_000138.5?content-type=application%2Fjson
https://rest.

In [18]:
i1 = individuals[0]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

{
  "id": "B7",
  "subject": {
    "id": "B7",
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P13Y"
      }
    },
    "sex": "FEMALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0000098",
        "label": "Tall stature"
      },
      "onset": {
        "age": {
          "iso8601duration": "P13Y"
        }
      }
    },
    {
      "type": {
        "id": "HP:0001166",
        "label": "Arachnodactyly"
      },
      "onset": {
        "age": {
          "iso8601duration": "P13Y"
        }
      }
    },
    {
      "type": {
        "id": "HP:0000768",
        "label": "Pectus carinatum"
      },
      "onset": {
        "age": {
          "iso8601duration": "P13Y"
        }
      }
    },
    {
      "type": {
        "id": "HP:0000218",
        "label": "High palate"
      },
      "onset": {
        "age": {
          "iso8601duration": "P13Y"
        }
      }
    },
    {
      "type": {
        "id": "HP:0025573",
        "l

In [19]:
output_directory = "phenopackets"
encoder.output_phenopackets(outdir=output_directory)

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3973G>T/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3037G>A/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.4955G>A/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3344A>G/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3713A>G/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3058A>G/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3898A>G/NM_000138.5?content-type=application%2Fjson
https://rest.